#### Task for today<br>
Dataset https://archive.ics.uci.edu/ml/datasets/Bag+of+Words <br>
Download last 5 txt files in data folder<br><br>
Q1. Try to find out a count of each and every word in a respective file and list of tuples with word and its respective count <br>&emsp;&emsp;Ex: word abort has occured how many times in that file<br>
&emsp;&emsp;Sample example: [('sudh', 6), ('kumar', 3)] <br>
Q2. Try to perform a reduce operation to get a count of all the words starting with the same alphabet<br>
&emsp;&emsp;Sample example: [('a', 6), ('b', 34), ......] <br>
Q3. Try to filter out all the words from dataset<br>
&emsp;&emsp;Sample example: .001.abstract = abstract<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&ensp;=.002 = delete<br>
&emsp;&emsp;and wherever there is a number don't extract that<br>
Q4. Create a tuple set of all the records available in all the five files and then sore it in sqlite DB.<br>
&emsp;&emsp;Sample example: (aah, >=, 354, fdsf, wer)<br> &emsp;&emsp;**HINT: 1. Use zip function<br>&emsp;&emsp;&emsp;&emsp;&emsp;2. Use for loop over a list of tuples and insert it into DB<br>&emsp;&emsp;&emsp;&emsp;&emsp;3. list 1 is representing file no 1, list 2 is representing file no 2 and so on. Read out all the files and store it in a list.**

In [1]:
import os
import csv
import sqlite3
import re

#### Question 1

In [2]:
for file in os.listdir():
    if file.endswith('.txt'):
        db = sqlite3.connect(f'{file.split(".txt")[0]}.db')
        cursor = db.cursor()
        
        cursor.execute(f'CREATE TABLE IF NOT EXISTS words (words TEXT)')
        
        with open(file, encoding='utf-8') as f:
            reader = csv.reader(f)
            
            for word in reader:
                cursor.execute('INSERT INTO words VALUES (?)', (word[0],))
            
            db.commit()
    
        db.close()

In [3]:
for file in os.listdir():
    if file.startswith('vocab') and file.endswith('.db'):
        db = sqlite3.connect(file)
        cursor = db.cursor()
        
        cursor.execute('SELECT words, COUNT(*) FROM words GROUP BY words LIMIT 5')
        print(f'For database {file}: {cursor.fetchall()}')
        
        db.close()

For database vocab.enron.db: [('aaa', 1), ('aaas', 1), ('aactive', 1), ('aadvantage', 1), ('aaker', 1)]
For database vocab.kos.db: [('aarp', 1), ('abandon', 1), ('abandoned', 1), ('abandoning', 1), ('abb', 1)]
For database vocab.nips.db: [('a2i', 1), ('aaa', 1), ('aaai', 1), ('aapo', 1), ('aat', 1)]
For database vocab.nytimes.db: [('aah', 1), ('aahed', 1), ('aaron', 1), ('aback', 1), ('abacus', 1)]
For database vocab.pubmed.db: [('!kung', 1), ('#1', 1), ('#10', 1), ('#11', 1), ('#12', 1)]


#### Question 2

In [4]:
def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

for file in os.listdir():
    if file.startswith('vocab') and file.endswith('.db'):
        db = sqlite3.connect(file)
        db.create_function("REGEXP", 2, regexp)
        cursor = db.cursor()
        
        cursor.execute('SELECT SUBSTR(words, 1, 1) AS first_letter, COUNT(*) FROM words WHERE first_letter REGEXP "^[A-Za-z]" GROUP BY first_letter')
        print(f'For database {file}:') 
        print(f'{cursor.fetchall()}')
        
        db.close()

For database vocab.enron.db:
[('a', 1800), ('b', 1557), ('c', 2611), ('d', 1664), ('e', 1404), ('f', 1160), ('g', 859), ('h', 1000), ('i', 1084), ('j', 377), ('k', 426), ('l', 989), ('m', 1648), ('n', 768), ('o', 685), ('p', 1945), ('q', 135), ('r', 1723), ('s', 2826), ('t', 1366), ('u', 562), ('v', 469), ('w', 805), ('x', 62), ('y', 111), ('z', 66)]
For database vocab.kos.db:
[('a', 492), ('b', 368), ('c', 666), ('d', 448), ('e', 294), ('f', 333), ('g', 195), ('h', 246), ('i', 269), ('j', 89), ('k', 70), ('l', 257), ('m', 344), ('n', 139), ('o', 175), ('p', 525), ('q', 30), ('r', 461), ('s', 742), ('t', 323), ('u', 105), ('v', 96), ('w', 205), ('y', 26), ('z', 8)]
For database vocab.nips.db:
[('a', 852), ('b', 513), ('c', 1173), ('d', 704), ('e', 588), ('f', 458), ('g', 352), ('h', 354), ('i', 710), ('j', 107), ('k', 184), ('l', 475), ('m', 769), ('n', 352), ('o', 321), ('p', 910), ('q', 71), ('r', 779), ('s', 1351), ('t', 587), ('u', 230), ('v', 230), ('w', 225), ('x', 27), ('y', 47)

#### Question 3

In [5]:
for file in os.listdir():
    if file.endswith('.txt'):
        with open(file, encoding='utf-8') as f:
            words = f.read().split()
            
            print(f'For file {file}:')
            print(list(filter(lambda x: re.match('^[a-zA-Z]+$', x), words)))

For file vocab.enron.txt:
['aaa', 'aaas', 'aactive', 'aadvantage', 'aaker', 'aap', 'aapg', 'aaron', 'aarp', 'aas', 'aau', 'aba', 'abacus', 'abag', 'abalone', 'abandon', 'abandoned', 'abandoning', 'abandonment', 'abate', 'abated', 'abb', 'abbott', 'abbreviated', 'abbreviation', 'abby', 'abc', 'abcm', 'abdullah', 'abe', 'abel', 'aberdeen', 'abernathy', 'aberration', 'abeyance', 'abhay', 'abhyankar', 'abide', 'abilene', 'abilities', 'ability', 'abitibi', 'able', 'abn', 'abnamro', 'abnormal', 'aboard', 'abolish', 'abolished', 'abolishing', 'abortion', 'abound', 'aboutface', 'aboveaverage', 'abovemarket', 'abovereferenced', 'abraham', 'abramson', 'abreast', 'abroad', 'abrogate', 'abrogating', 'abrupt', 'abruptly', 'absence', 'absences', 'absense', 'absent', 'absentee', 'absolute', 'absolutely', 'absorb', 'absorbed', 'absorbing', 'absorption', 'abstain', 'abstract', 'absurd', 'abt', 'abu', 'abundance', 'abundant', 'abundantly', 'abuse', 'abused', 'abuses', 'abusing', 'abusive', 'abx', 'aca',

#### Question 4

In [6]:
file_lists = []

for file in os.listdir():
    if file.endswith('.txt'):
        with open(file, encoding='utf-8') as f: 
            file_lists.append(f.read().split())
            
records = list(zip(*file_lists))

db = sqlite3.connect('concat.db')
cursor = db.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS concat(
    file1 TEXT, 
    file2 TEXT,
    file3 TEXT,
    file4 TEXT,
    file5 TEXT
)''')

for record in records:
    cursor.execute('INSERT INTO concat VALUES(?, ?, ?, ?, ?)', record)
    
db.commit()

In [7]:
cursor.execute('SELECT * FROM concat')
cursor.fetchall()

[('aaa', 'aarp', 'a2i', 'aah', '>='),
 ('aaas', 'abandon', 'aaa', 'aahed', '>>'),
 ('aactive', 'abandoned', 'aaai', 'aaron', '>>>'),
 ('aadvantage', 'abandoning', 'aapo', 'aback', '>/='),
 ('aaker', 'abb', 'aat', 'abacus', '->'),
 ('aap', 'abc', 'aazhang', 'abajo', '--'),
 ('aapg', 'abcs', 'abandonment', 'abalone', '-->'),
 ('aaron', 'abdullah', 'abbott', 'abandon', '-/-'),
 ('aarp', 'ability', 'abbreviated', 'abandoned', '-/+'),
 ('aas', 'aboard', 'abcde', 'abandoning', '/-'),
 ('aau', 'abortion', 'abe', 'abandonment', '/+-'),
 ('ab1890', 'abortions', 'abeles', 'abandono', '..'),
 ('ab1x', 'abraham', 'abi', 'abarnard', '...'),
 ('ab31x', 'abrams', 'abilistic', 'abashed', '+-'),
 ('aba', 'abroad', 'abilities', 'abate', '+/'),
 ('abacus', 'absence', 'ability', 'abated', '+/--'),
 ('abag', 'absent', 'abl', 'abatement', '+/?'),
 ('abalone', 'absentee', 'able', 'abating', '+/+'),
 ('abandon', 'absolute', 'ables', 'abbey', '++'),
 ('abandoned', 'absolutely', 'ablex', 'abbot', '+++'),
 ('aba